In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.metrics import dtw

In [1]:
import polars as pl

In [ ]:
data=pl.read_csv('dns_oct-jan.csv')

In [11]:
data_small = pl.read_csv('3months_under10_000.csv', try_parse_dates = True)

In [12]:
data_small.head()

shieldid,day,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois
str,date,str,str,i64,i64,datetime[μs],datetime[μs],i64,i64,f64,str,date,date,date,str
"""enilokmacy""",2023-01-04,"""1334519-19.cha…","""drift.com""",1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,"""drift.com""",1995-04-15,1995-04-15,2024-04-16,"""whois.ausregis…"
"""enilokmacy""",2023-01-10,"""istio.api.drif…","""drift.com""",1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,"""drift.com""",1995-04-15,1995-04-15,2024-04-16,"""whois.ausregis…"
"""fecjavfec""",2023-01-09,"""1500556-6.chat…","""drift.com""",1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,"""drift.com""",1995-04-15,1995-04-15,2024-04-16,"""whois.ausregis…"
"""fecjavfec""",2023-01-20,"""metrics.api.dr…","""drift.com""",4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,"""drift.com""",1995-04-15,1995-04-15,2024-04-16,"""whois.ausregis…"
"""oreethecaw""",2023-01-20,"""presence.api.d…","""drift.com""",15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,"""drift.com""",1995-04-15,1995-04-15,2024-04-16,"""whois.ausregis…"


In [4]:
data_small.whois.fillna('unknown',inplace=True)

In [5]:
# create a timeseen column
data_small['timelastseen'] = pd.to_datetime(data_small['timelastseen'])

In [6]:
data_small['timefirstseen'] = pd.to_datetime(data_small['timefirstseen'])

In [7]:
data_small['timeseen'] = (data_small['timelastseen']-data_small['timefirstseen'])

In [8]:
data_small['timeseen'] = data_small['timeseen'].dt.total_seconds()

### Find and create an anomaly column with Finns method

In [9]:
#organize data like in linear regression example
counts = data_small.pivot_table(values = 'num_clients',index = data_small.index.date,
                         columns = 'dns_qdomainname',
                         aggfunc=sum,
                         fill_value = 0)

In [10]:
#normalize data
counts.max().sort_values(ascending=False).to_csv('counts.csv')
counts = counts/counts.max()

In [11]:
#create domians by checking similairty to 2 anomalies
domains = []
dns = data_small.dns_qdomainname.unique()
similarity_slider = .6
#numba this
for i in range (len(dns)-1):
    val = dtw(counts['beside.media'],counts[dns[i]])
    if (val <= similarity_slider) & (dtw(counts['123w0w.com'],counts[dns[i]]) <= similarity_slider):
        domains.append(dns[i])

In [12]:
len(domains)

1161

In [13]:
data_small.loc[:,'anomaly_2'] = 0
data_small.loc[data_small.dns_qdomainname.isin(domains),'anomaly_2'] = 1

In [14]:
data_small.head()

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois,timeseen,anomaly_2
day,,,,,,,,,,,,,,,,,
2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0.000,0
2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0.000,0
2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,0.000,0
2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,38776.076,0
2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au,41656.222,0


### Create a column of tld's and whether or not it is known to be malicious

In [15]:
# tld column
data_small['tld'] = data_small.dns_qdomainname.str.split('.').str[1]

In [16]:
# column of tlds cross referenced with a list of the top tlds used in malicious activity
mal_tlds = ['live','monster','okinawa','top','boutique','beauty','cn','zone','autos','zw','bd','ke','am','sbs','date','pw','quest','cd','bid','cyou','ml','ws','icu','stream','email','uno','cam','xyz','tokyo','help','casa','rest','win']
data_small.loc[:,'mal_tld']=0
data_small.loc[data_small.tld.isin(mal_tlds),'mal_tld']=1
data_small.mal_tld.sum()

36736

In [17]:
# of the potentialy anomalys using anomaly finder 2, how many fall into the list of malicious tld
domains
domains_tld=[i.split('.')[1] for i in domains]
domains_mal = set(domains_tld) & set(mal_tlds)
domains_mal

{'cn', 'email', 'live', 'top', 'xyz'}

In [18]:
# anomaly finder 1?
anomaly1 = data_small.loc[data_small.anomaly==1].dns_qdomainname.unique().tolist()
anomaly1_tld = [i.split('.')[1] for i in domains]
domains1_mal = set(anomaly1_tld) & set(mal_tlds)
domains1_mal

{'cn', 'email', 'live', 'top', 'xyz'}

### Fixing the bad data in the created, updated, expired columns

In [41]:
data_small.reset_index(inplace=True)


In [20]:
# get the indices of all bad times
for i in range(len(data_small)):
    try:
        pd.to_datetime(data_small.expires[i])
    except:
        print(i)


7017679
7017680
7017681
7017682
7017683
7017684
7017685
7017686
7017687
7017688
7017689
7017690
7017691
7017692
7017693
7017694
7017695
7017696
7017697
7017698
7017699
7017700
7017701
7017702
7017703
7017704
7017705
7017706
7017707
7017708
7336087
7336088
7336089
7336090
7336091
7336092
7336093
7336094
7336095
7336096
7336097
7336098
7336099
7336100
7336101
7336102
7336103
7336104
7336105
7336106
7336107
7336108
7336109
7336110
7336111
7336112
7336113
7336114
7336115
7336116
7336117
7336118
7336119
7336120
7336121
7336122
7336123
7336124
7336125
7336126
7336127
7336128
7336129
7336130
7336131
7336132
7336133
7336134
7336135
7336136
7336137
7336138
7336139
7336140
7336141
7336142
7336143
7336144
7336145
7336146
7336147
7336148
7336149
7336150
7336151
7336152
7336153
7336154
7336155
7628174
7628175
7628176
7628177
7628178
7628179
7628180
7628181
7628182
7628183
7628184
7628185
7628186
7628187
7647656
7647657
7647658
7647659
7647660
7647661
7647662
7647663
7647664
7647665
7647666
7647667


In [21]:
badtime_e = [7017679,
7017680,
7017681,
7017682,
7017683,
7017684,
7017685,
7017686,
7017687,
7017688,
7017689,
7017690,
7017691,
7017692,
7017693,
7017694,
7017695,
7017696,
7017697,
7017698,
7017699,
7017700,
7017701,
7017702,
7017703,
7017704,
7017705,
7017706,
7017707,
7017708,
7336087,
7336088,
7336089,
7336090,
7336091,
7336092,
7336093,
7336094,
7336095,
7336096,
7336097,
7336098,
7336099,
7336100,
7336101,
7336102,
7336103,
7336104,
7336105,
7336106,
7336107,
7336108,
7336109,
7336110,
7336111,
7336112,
7336113,
7336114,
7336115,
7336116,
7336117,
7336118,
7336119,
7336120,
7336121,
7336122,
7336123,
7336124,
7336125,
7336126,
7336127,
7336128,
7336129,
7336130,
7336131,
7336132,
7336133,
7336134,
7336135,
7336136,
7336137,
7336138,
7336139,
7336140,
7336141,
7336142,
7336143,
7336144,
7336145,
7336146,
7336147,
7336148,
7336149,
7336150,
7336151,
7336152,
7336153,
7336154,
7336155,
7628174,
7628175,
7628176,
7628177,
7628178,
7628179,
7628180,
7628181,
7628182,
7628183,
7628184,
7628185,
7628186,
7628187,
7647656,
7647657,
7647658,
7647659,
7647660,
7647661,
7647662,
7647663,
7647664,
7647665,
7647666,
7647667,
7647668,
7647669,
7647670,
7647671,
7647672,
7647673,
7647674,
7647675,
7647676,
7647677,
7647678,
7647679,
7647680,
7647681,
7647682,
7647683,
7647684,
7647685,
7647686,
7647687,
7647688,
7647689,
7647690,
7647691,
7647692,
7647693,
7647694,
7647695,
7647696,
7647697,
7647698,
7647699,
7647700,
7647701,
7647702,
7647703,
7647704,
7647705,
7647706,
7647707,
7647708,
7794828,
7794829,
7794830,
7794831,
7794832,
7794833,
7794834,
7794835,
7794836,
7794837,
7846575,
7846576,
7846577,
7846578,
7846579,
7846580,
7846581,
7846582,
7846583,
7846584,
7853177,
7903029,
7938764,
7938765,
7938766,
7974129,
7974130,
7974131,
7974132,
7974133,
7974134,
7991433,
7991434,
8021368,
8021369,
8021370,
8021371,
8021372,
8021373,
8021374,
8021375,
8021376,
8021377,
8021378,
8021379,
8021380,
8048752,
8048753,
8048754,
8048755,
8048756,
8048757,
8057153,
8069148,
8078112,
8116554,
8116555,
8135395,
8138031,
8141359,
8160724,
8160725,
8179615,
8189456,
8193134,
8203184,
8203185,
8205323,
8205449,
8205450,
8208303,
8222717,
8222718,
8224212,
8227575,
8229089,
8257853,
8264014]

In [22]:
badtime=[7017679,
7017680,
7017681,
7017682,
7017683,
7017684,
7017685,
7017686,
7017687,
7017688,
7017689,
7017690,
7017691,
7017692,
7017693,
7017694,
7017695,
7017696,
7017697,
7017698,
7017699,
7017700,
7017701,
7017702,
7017703,
7017704,
7017705,
7017706,
7017707,
7017708,
7336087,
7336088,
7336089,
7336090,
7336091,
7336092,
7336093,
7336094,
7336095,
7336096,
7336097,
7336098,
7336099,
7336100,
7336101,
7336102,
7336103,
7336104,
7336105,
7336106,
7336107,
7336108,
7336109,
7336110,
7336111,
7336112,
7336113,
7336114,
7336115,
7336116,
7336117,
7336118,
7336119,
7336120,
7336121,
7336122,
7336123,
7336124,
7336125,
7336126,
7336127,
7336128,
7336129,
7336130,
7336131,
7336132,
7336133,
7336134,
7336135,
7336136,
7336137,
7336138,
7336139,
7336140,
7336141,
7336142,
7336143,
7336144,
7336145,
7336146,
7336147,
7336148,
7336149,
7336150,
7336151,
7336152,
7336153,
7336154,
7336155,
7628174,
7628175,
7628176,
7628177,
7628178,
7628179,
7628180,
7628181,
7628182,
7628183,
7628184,
7628185,
7628186,
7628187,
7846575,
7846576,
7846577,
7846578,
7846579,
7846580,
7846581,
7846582,
7846583,
7846584,
7853177,
7903029,
7938764,
7938765,
7938766,
7974129,
7974130,
7974131,
7974132,
7974133,
7974134,
7991433,
7991434,
8021368,
8021369,
8021370,
8021371,
8021372,
8021373,
8021374,
8021375,
8021376,
8021377,
8021378,
8021379,
8021380,
8048752,
8048753,
8048754,
8048755,
8048756,
8048757,
8057153,
8069148,
8078112,
8094647,
8116554,
8116555,
8135395,
8138031,
8141359,
8160724,
8160725,
8179615,
8189456,
8193134,
8203184,
8203185,
8205323,
8205449,
8205450,
8208303,
8222717,
8222718,
8224212,
8227575,8229089,8257853,8264014]

In [34]:
all_badtime = list(set(badtime) | set(badtime_e))
len(all_badtime)

245

In [42]:
badtime_ind = data_small.index[all_badtime]
badtime_cols = data_small.iloc[all_badtime,:]

In [36]:
# are any of the domains with bad times in our list projected anomalies?
badtime_doms = badtime_cols.dns_qdomainname.unique()
set(badtime_doms)&set(domains)

set()

In [37]:
badtime_cols

,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois,timeseen,anomaly_2,tld,mal_tld
day,,,,,,,,,,,,,,,,,,,
2022-12-29,myakitjit,sionnet.rs,sionnet.rs,1,2,2022-12-29 03:03:16.803,2022-12-29 03:03:17.132,0,1,0.0,sionnet.rs,0009-09-29,0009-09-29,0009-10-12,unknown,0.329,0,rs,0
2023-01-11,enilokmacy,iss.rs,iss.rs,3,6,2023-01-11 16:33:49.891,2023-01-11 16:33:49.958,0,1,0.0,iss.rs,0031-11-29,0031-11-29,0031-12-12,unknown,0.067,0,rs,0
2022-12-12,scefunnu,workandtravel.rs,workandtravel.rs,2,2,2022-12-12 18:54:25.471,2022-12-12 18:54:25.500,0,1,0.0,workandtravel.rs,0032-05-01,0032-05-01,0032-05-13,unknown,0.029,0,rs,0
2022-12-13,fecjavfec,www.manua.ls,manua.ls,1,1,2022-12-13 21:03:32.620,2022-12-13 21:03:32.620,0,1,0.0,manua.ls,0028-04-11,0028-04-11,0028-04-12,unknown,0.000,0,ls,0
2023-01-04,fecjavfec,www.manua.ls,manua.ls,2,4,2023-01-04 13:18:34.451,2023-01-04 18:22:48.192,0,1,0.0,manua.ls,0028-04-11,0028-04-11,0028-04-12,unknown,18253.741,0,ls,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-16,rhibeicdyb,154.205.20.234.1673866800.main.research.openre...,openresolve.rs,1,1,2023-01-16 16:09:43.450,2023-01-16 16:09:43.450,0,1,0.0,openresolve.rs,0019-06-11,0019-06-11,0019-06-13,unknown,0.000,0,rs,0
2023-01-16,rhibeicdyb,154-205-20-234.1673888074.main.research.openre...,openresolve.rs,2,2,2023-01-16 17:00:48.632,2023-01-16 17:00:48.636,0,1,0.0,openresolve.rs,0019-06-11,0019-06-11,0019-06-13,unknown,0.004,0,rs,0
2023-01-23,rhibeicdyb,154-205-20-234.1674499805.anyc.research.openre...,openresolve.rs,1,2,2023-01-23 19:01:29.512,2023-01-23 19:01:29.512,0,1,0.0,openresolve.rs,0019-06-11,0019-06-11,0019-06-13,unknown,0.000,0,rs,0


In [48]:
#badtime_dates
len(data_small)

8267459

In [50]:
data_small = data_small.drop(badtime_ind, axis=0)

In [51]:
len(data_small)

8267214

In [52]:
data_small.anomaly_2.sum()

5487

In [53]:
data_small['created'] = pd.to_datetime(data_small['created'])
data_small['updated'] = pd.to_datetime(data_small['updated'])
data_small['expires'] = pd.to_datetime(data_small['expires'])

In [54]:
data_small['time_existed'] = data_small['expires'] - data_small['created']
data_small['time_existed']

0         10594 days
1         10594 days
2         10594 days
3         10594 days
4         10594 days
             ...    
8267454     365 days
8267455    8035 days
8267456   10957 days
8267457    1461 days
8267458          NaT
Name: time_existed, Length: 8267214, dtype: timedelta64[ns]

In [68]:
data_small['time_existed'] = data_small.time_existed.astype(str)

In [69]:
data_small['time_existed'] = data_small.time_existed.str.split(' ').str[0]

In [70]:
data_small.rename(columns = {'time_existed':'days_existed'}, inplace = True)

In [76]:
data_small['days_existed'] = data_small.days_existed.replace('NaT', np.nan)

In [77]:
data_small['days_existed'] = data_small.days_existed.astype(pd.Int64Dtype())

In [78]:
data_small.anomaly_2.sum()

5487

### convert timefirstseen, timelastseen, created, expires columns to numeric values

In [79]:
data_small['month_firstseen'] = data_small.timefirstseen.dt.month
data_small['day_firstseen'] = data_small.timefirstseen.dt.weekday
data_small['hour_firstseen'] = data_small.timefirstseen.dt.hour
data_small['month_lastseen'] = data_small.timelastseen.dt.month
data_small['day_lastseen'] = data_small.timelastseen.dt.weekday
data_small['hour_lastseen'] = data_small.timelastseen.dt.hour
data_small['day_created'] = data_small.created.dt.weekday
data_small['month_created'] = data_small.created.dt.month
data_small['day_expires'] = data_small.expires.dt.weekday
data_small['month_expires'] = data_small.expires.dt.month

In [81]:
data_small.to_csv('data_small_update.csv')

In [82]:
data_small.head()

,day,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,...,month_firstseen,day_firstseen,hour_firstseen,month_lastseen,day_lastseen,hour_lastseen,day_created,month_created,day_expires,month_expires
0,2023-01-04,enilokmacy,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,...,1,2,0,1,2,0,5.0,4.0,1.0,4.0
1,2023-01-10,enilokmacy,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,...,1,1,13,1,1,13,5.0,4.0,1.0,4.0
2,2023-01-09,fecjavfec,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,...,1,0,15,1,0,15,5.0,4.0,1.0,4.0
3,2023-01-20,fecjavfec,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,...,1,4,10,1,4,20,5.0,4.0,1.0,4.0
4,2023-01-20,oreethecaw,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,...,1,4,2,1,4,14,5.0,4.0,1.0,4.0


## Random Forest Classifier (anomaly_2)

In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.impute import SimpleImputer

In [ ]:
data_small.head()

In [ ]:
data_small.columns

In [ ]:
X = data_small.drop(['day','anomaly','anomaly_2','dns_qname','dns_qdomainname','timefirstseen','timelastseen','domain','created','updated','expires'],axis=1)
y = data_small['anomaly_2']

In [ ]:
data_small.isna().sum()

In [ ]:
X.columns

In [ ]:
numerical_feats = ['num_clients', 'num_responses','shieldid_count','rank','timeseen',
                   'days_existed','day_firstseen','hour_firstseen','month_firstseen', 
                   'month_lastseen', 'day_lastseen','hour_lastseen', 'day_created',
                   'month_created', 'day_expires','month_expires']
categorical_feats = ['shieldid','whois','tld','mal_tld']

In [ ]:
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler())  
])

cat_processor = Pipeline(steps=[
     ('imputer', SimpleImputer(strategy = 'constant')),
     ('encoder', OneHotEncoder())   
 ])

In [ ]:
feat_processor = ColumnTransformer(transformers = [
    ('num_pipe', num_processor,numerical_feats),
    ('cat_pipe', cat_processor, categorical_feats)
],remainder='drop')

In [90]:
RF= RandomForestClassifier()

In [ ]:
pipe = Pipeline(steps=[
    ('feature_processor', feat_processor),
    ('clf', RF)
])
pipe

In [ ]:
pipe.fit(X,y)

In [ ]:
ypred=best_clf.predict(xtest)
yproba=best_clf.predict_proba(xtest)
yproba=np.array(yproba)
accuracy_score(ypred,y_test)

## Random Forest (anomaly)

In [83]:
X = data_small.drop(['day','anomaly','anomaly_2','dns_qname','dns_qdomainname','timefirstseen','timelastseen','domain','created','updated','expires'],axis=1)
y = data_small['anomaly']

In [84]:
numerical_feats = ['num_clients', 'num_responses','shieldid_count','rank','timeseen',
                   'days_existed','day_firstseen','hour_firstseen','month_firstseen', 
                   'month_lastseen', 'day_lastseen','hour_lastseen', 'day_created',
                   'month_created', 'day_expires','month_expires']
categorical_feats = ['shieldid','whois','tld','mal_tld']

In [87]:
num_processor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler())  
])

cat_processor = Pipeline(steps=[
     ('imputer', SimpleImputer(strategy = 'constant')),
     ('encoder', OneHotEncoder())   
 ])

In [88]:
feat_processor = ColumnTransformer(transformers = [
    ('num_pipe', num_processor,numerical_feats),
    ('cat_pipe', cat_processor, categorical_feats)
],remainder='drop')

In [91]:
pipe2 = Pipeline(steps=[
    ('feature_processor', feat_processor),
    ('clf', RF)
])
pipe2

Pipeline(steps=[('feature_processor',
                 ColumnTransformer(transformers=[('num_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['num_clients',
                                                   'num_responses',
                                                   'shieldid_count', 'rank',
                                                   'timeseen', 'days_existed',
                                                   'day_firstseen',
                                                   'hour_firstseen',
                                                   'month_firstseen',
                                                   'month_lastseen',
                                                   'day_lastseen',
                                                   'hour_lastseen',
                                                   'day_created',
                                                   'month_created',
                                                   'day_expires',
                                                   'month_expires']),
                                                 ('cat_pipe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['shieldid', 'whois', 'tld',
                                                   'mal_tld'])])),
                ('clf', RandomForestClassifier())])

In [ ]:
pipe2.fit(X,y)